In this article we’ll go over the process of analysing an A/B experiment, from formulating a hypothesis, testing it, and finally interpreting results. https://www.kaggle.com/datasets/zhangluyuan/ab-testing/versions/1?resource=download

Let’s imagine you work on the product team at a medium-sized online e-commerce business. The UX designer worked really hard on a new version of the product page, with the hope that it will lead to a higher conversion rate. The product manager (PM) told you that the current conversion rate is about 13% on average throughout the year, and that the team would be happy with an increase of 2%, meaning that the new design will be considered a success if it raises the conversion rate to 15%.

In [1]:
import pandas as pd
from numpy import percentile
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import seaborn as sns
sns.set_style("whitegrid")
import warnings
warnings.filterwarnings('ignore')

import scipy.stats as stats
from scipy import stats as st
import statsmodels.stats.api as sms
from math import ceil
from scipy.stats import shapiro
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.graphics.factorplots import interaction_plot

In [2]:
df = pd.read_csv("ab_data.csv")
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [3]:
df.shape

(294478, 5)

In [4]:
df.info

<bound method DataFrame.info of         user_id                   timestamp      group landing_page  converted
0        851104  2017-01-21 22:11:48.556739    control     old_page          0
1        804228  2017-01-12 08:01:45.159739    control     old_page          0
2        661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3        853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4        864975  2017-01-21 01:52:26.210827    control     old_page          1
...         ...                         ...        ...          ...        ...
294473   751197  2017-01-03 22:28:38.630509    control     old_page          0
294474   945152  2017-01-12 00:51:57.078372    control     old_page          0
294475   734608  2017-01-22 11:45:03.439544    control     old_page          0
294476   697314  2017-01-15 01:20:28.957438    control     old_page          0
294477   715931  2017-01-16 12:40:24.467417  treatment     new_page          0

[294478 rows x 5 co

In [5]:
df.describe

<bound method NDFrame.describe of         user_id                   timestamp      group landing_page  converted
0        851104  2017-01-21 22:11:48.556739    control     old_page          0
1        804228  2017-01-12 08:01:45.159739    control     old_page          0
2        661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3        853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4        864975  2017-01-21 01:52:26.210827    control     old_page          1
...         ...                         ...        ...          ...        ...
294473   751197  2017-01-03 22:28:38.630509    control     old_page          0
294474   945152  2017-01-12 00:51:57.078372    control     old_page          0
294475   734608  2017-01-22 11:45:03.439544    control     old_page          0
294476   697314  2017-01-15 01:20:28.957438    control     old_page          0
294477   715931  2017-01-16 12:40:24.467417  treatment     new_page          0

[294478 rows x 5 

In [6]:
df.pivot_table(index=['group','landing_page'], values='converted', aggfunc='mean')

converted
group     landing_page           
control   new_page       0.121369
          old_page       0.120386
treatment new_page       0.118807
          old_page       0.127226

In [7]:
dups = df.duplicated()
print(dups.any())

False


In [8]:
df[df["user_id"].duplicated()]

,user_id,timestamp,group,landing_page,converted
2656,698120,2017-01-15 17:13:42.602796,control,old_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0
7500,899953,2017-01-07 03:06:54.068237,control,new_page,0
8036,790934,2017-01-19 08:32:20.329057,treatment,new_page,0
10218,633793,2017-01-17 00:16:00.746561,treatment,old_page,0
...,...,...,...,...,...
294308,905197,2017-01-03 06:56:47.488231,treatment,new_page,0
294309,787083,2017-01-17 00:15:20.950723,control,old_page,0
294328,641570,2017-01-09 21:59:27.695711,control,old_page,0
294331,689637,2017-01-13 11:34:28.339532,control,new_page,0


In [9]:
# Check if there is mismatch between group and landing_page
df_mismatch = df[(df["group"]=="treatment")&(df["landing_page"]=="old_page")
                |(df["group"]=="control")&(df["landing_page"]=="new_page")]
n_mismatch = df_mismatch.shape[0]
print(f"The number of mismatched rows:{n_mismatch} rows" )
print("Percent of mismatched rows:%.2f%%" % (n_mismatch/df.shape[0]*100))

The number of mismatched rows:3893 rows
Percent of mismatched rows:1.32%


As you can see, there are 3893 rows mismatched, which means that treatment group mismatched with old_page and control group mismatched with new_page.There are also 3895 duplicated user_id, some of them loading both new page and old page. This might be the relative department repeat the expermient after finding the mismatched problem. Therefore, we need to only keep the right data and drop the mismatched data.

In [10]:
df2 = df[(df["group"]=="treatment")&(df["landing_page"]=="new_page")
                |(df["group"]=="control")&(df["landing_page"]=="old_page")]

In [11]:
# Check duplicate user_id again
df2[df2["user_id"].duplicated()]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [12]:
df2.pivot_table(index=['group','landing_page'], values='converted', aggfunc='mean')

,,converted
group,landing_page,
control,old_page,0.120386
treatment,new_page,0.118807


In [13]:
df2.drop_duplicates(subset="user_id",inplace=True) 

In [14]:
df2.shape

(290584, 5)

In [15]:
null_hypothesis = 'Perubahan Landing Page Tidak Berpengaruh Terhadap Converted'
alternate_hypothesis = 'Perubahan Landing Page Berpengaruh Terhadap Converted'

In [16]:
# comparing two proportions
effect_size = sms.proportion_effectsize(0.13, 0.15)   
# solve for any one parameter of the power of a two sample z-test
required_n = sms.NormalIndPower().solve_power(effect_size, power=0.8,  alpha=0.05,ratio=1)                                                 
required_n = ceil(required_n)                                                   

print(required_n)

4720


In [17]:
control_sample = df2[df2['group'] == 'control'].sample(n=required_n, random_state=22)
treatment_sample = df2[df2['group'] == 'treatment'].sample(n=required_n, random_state=22)

ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [18]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'ci 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'ci 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

p_value = f'p-value: {pval:.3f}'

z statistic: 0.38
p-value: 0.703
ci 95% for control group: [0.111, 0.130]
ci 95% for treatment group: [0.109, 0.127]


In [19]:
if p_value < '0.05':
    print('the null hypothesis should be rejected and accept alternate hypothesis')
    print(alternate_hypothesis)
else:
    print('the alternate hypothesis should be rejected and accept null hypothesis')
    print(null_hypothesis)

the alternate hypothesis should be rejected and accept null hypothesis
Perubahan Landing Page Tidak Berpengaruh Terhadap Converted
